### Outline of GPICL
Dataset $D = \{x_i, y_i\}$, linear projection $A \in \mathcal{R}^{N_x \times N_x}$ with $A_{ij} \sim N(0, 1/N_x)$ and final output permutation $\rho$ , $D = \{Ax_i, \rho(y_i)\}$.

this is done to reduce the amount of unique tasks necessary to train our meta-learned model. The loss used is cross-entropy loss, between the label y_j and prediction on the entire series except for the last label. Essentially the same the other notebook leaving out only the last label and adding the set of samples as context.

The model a ViT with learned positional embeddings.